In [1]:
import requests
import zipfile
import os
import sys
import io
import pandas as pd
from io import BytesIO
from tqdm import tqdm
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import logging
import time
import geopandas as gpd
import folium

In [2]:
main_path=os.path.dirname(os.path.abspath(os.getcwd()))
data_path = os.path.join(main_path, 'data', 'interim')
censo_path=os.path.join(data_path,'censo')
shp_path=os.path.join(data_path,'shp')

In [3]:
censo_ent_path=os.path.join(censo_path,'censo_ent_tidy_data.csv')
censo_mun_path=os.path.join(censo_path,'censo_mun_tidy_data.csv')

In [4]:
shp_ent_path=os.path.join(shp_path,'shp_ent_tidy_data.csv')
shp_mun_path=os.path.join(shp_path,'shp_mun_tidy_data.csv')

In [5]:
df_cve_ent = pd.read_csv(censo_ent_path, encoding='utf-8')
df_cve_mun = pd.read_csv(censo_mun_path, encoding='utf-8')

In [6]:
df_cve_ent.head()

,cvegeo,pob_tot,rel_h_m
0,1,1425607,95.58
1,2,3769020,101.72
2,3,798447,103.39
3,4,928363,96.93
4,5,3146771,98.77


In [7]:
df_cve_mun.head()

,cvegeo,pob_tot,rel_h_m
0,1001,948990,94.90
1,1002,51536,96.14
2,1003,58250,96.21
3,1004,17000,95.22
4,1005,129929,97.73


In [35]:
gdf_ent = gpd.read_file(shp_ent_path) 
gdf_mun = gpd.read_file(shp_mun_path) 

In [39]:
df_cve_mun.head()

,cvegeo,pob_tot,rel_h_m
0,1001,948990,94.90
1,1002,51536,96.14
2,1003,58250,96.21
3,1004,17000,95.22
4,1005,129929,97.73


In [38]:
gdf_mun_pob = gdf_mun.merge(df_cve_mun, on=['cvegeo'], how='left')
gdf_mun_pob=gdf_mun_pob[['cvegeo','nomgeo','pob_tot','rel_h_m','geometry']] # desired order
gdf_mun_pob.head()

ValueError: You are trying to merge on object and int64 columns for key 'cvegeo'. If you wish to proceed you should use pd.concat

In [ ]:
gdf_ent_pob = gdf_ent.merge(df_cve_ent, on=['CVEGEO'], how='left')
gdf_ent_pob.drop(columns=['CVE_ENT', 'CVE_MUN'], errors='ignore', inplace=True)
gdf_ent_pob=gdf_ent_pob[['CVEGEO','NOMGEO','pob_tot','rel_h_m','geometry']] # desired order
gdf_ent_pob.head()

In [ ]:
# Define the style function to color the polygons based on rel_h_m values
def style_function(feature):
    # Convert rel_h_m to numeric, coerce errors to NaN
    rel_h_m_value = pd.to_numeric(feature['properties']['rel_h_m'], errors='coerce')

    if pd.notna(rel_h_m_value):  # Check if the value is not NaN
        if 99 <= rel_h_m_value <= 101:
            fill_color = 'green'
        elif rel_h_m_value > 101:
            fill_color = 'blue'
        else:
            fill_color = 'pink'
    else:
        fill_color = 'gray'  # Default color for NaN values

    return {
        'fillColor': fill_color,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.5,
    }

In [ ]:
# Create a folium map
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)  # Center the map on Mexico

# Add polygons to the map
folium.GeoJson(
    gdf_ent_pob,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['NOMGEO', 'pob_tot', 'rel_h_m'],  # pob_tot is the column with the population data
        aliases=['Entity:', 'Population:', 'Male per 100 Female:'],
        localize=True
    )
).add_to(m)

# Display the map
m.save('interactive_map.html')
m

In [ ]:
# Create a folium map
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)  # Center the map on Mexico, adjust location as needed

# Add polygons to the map
folium.GeoJson(
    gdf_mun_pob,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['NOMGEO','pob_tot','rel_h_m'],  # pob_tot is the column with the population data
        aliases=['Municipality:','Population:','Male per 100 Female:'], # #Result of dividing the total number of men by the total number of women
                                                                  # and multiplying it by one hundred. Indicates the number of men per 100 women.
        localize=True
    )
).add_to(m)

# Display the map
m.save('interactive_map.html')
m